In [192]:
from ncmodules import files
import pandas as pd
import plotly.express as px

In [193]:
# fig = px.scatter(x=[0, 1, 2, 3, 4], y=[0, 1, 4, 9, 16])
# fig.show()

In [194]:
dbname = "668.db"
logtable = "log_table" # table in nnn.db
df = files.read_database(dbname, logtable, "descending")

pd.set_option('display.max_columns', None)
print(df)

>> Displaying 668.db values:

    dive            time_start              time_end  gps_lat_start  \
0    250  2025-08-11T15:05:46Z  2025-08-11T19:41:33Z      19.875050   
1    249  2025-08-11T10:28:34Z  2025-08-11T15:01:10Z      19.886750   
2    248  2025-08-11T05:53:48Z  2025-08-11T10:22:34Z      19.901500   
3    247  2025-08-11T01:09:45Z  2025-08-11T05:49:09Z      19.911383   
4    246  2025-08-10T20:19:30Z  2025-08-11T01:04:53Z      19.921650   
5    245  2025-08-10T15:27:22Z  2025-08-10T20:10:22Z      19.925867   
6    244  2025-08-10T10:42:48Z  2025-08-10T15:22:41Z      19.930933   
7    243  2025-08-10T05:54:03Z  2025-08-10T10:38:03Z      19.942600   
8    242  2025-08-10T01:13:33Z  2025-08-10T05:49:38Z      19.954350   
9    241  2025-08-09T20:28:24Z  2025-08-10T01:08:59Z      19.964783   
10   240  2025-08-09T15:45:55Z  2025-08-09T20:24:03Z      19.976467   
11   239  2025-08-09T11:02:34Z  2025-08-09T15:41:36Z      19.984950   
12   238  2025-08-09T06:17:28Z  2025-08-09T10:5

In [195]:
df_hr_scaled = df["int_Humidity"]
df_p_scaled = df["int_Pressure"] * 7
df_t_scaled = df["int_Temperature"] * 3

health_plot = px.line(df, x="dive", y=[df_hr_scaled,df_p_scaled,df_t_scaled], range_x=[0,220])
health_plot.show()

# Subplots

In [196]:
from plotly.subplots import make_subplots
import plotly.graph_objects as go

In [197]:
# Create figure with subplots
health_fig = make_subplots(rows=6, cols=2, shared_xaxes=True, shared_yaxes=True, vertical_spacing=0.025, horizontal_spacing=0.01, column_widths=[0.9, 0.1])

# Add traces to figure
health_fig.add_trace(go.Scatter(x=df["dive"], y=df["int_Humidity"], mode="lines", line_color="red", name="int_Humidity"), row=1, col=1) # mode="lines+markers"
health_fig.add_trace(go.Box(y=df["int_Humidity"], name="", showlegend=False, line_color="red"), row=1, col=2)
health_fig.add_trace(go.Scatter(x=df["dive"], y=df["int_Pressure"], mode="lines", line_color="green", name="int_Pressure"), row=2, col=1)
health_fig.add_trace(go.Box(y=df["int_Pressure"], name="", showlegend=False, line_color="green"), row=2, col=2)
health_fig.add_trace(go.Scatter(x=df["dive"], y=df["int_Temperature"], mode="lines", line_color="blue", name="int_Temperature"), row=3, col=1)
health_fig.add_trace(go.Box(y=df["int_Temperature"], name="", showlegend=False, line_color="blue"), row=3, col=2)
health_fig.add_trace(go.Scatter(x=df["dive"], y=df["log_24_minv"], mode="lines", line_color="magenta", name="24_minv"), row=4, col=1)
health_fig.add_trace(go.Box(y=df["log_24_minv"], name="", showlegend=False, line_color="magenta"), row=4, col=2)
# mfig.add_trace(go.Scatter(x=df["dive"], y=df["log_10_minv"], mode="lines", line=dict(dash='dash', color="rgb(255,50,50)", width=1), name="10_minv"), row=4, col=1)
# mfig.add_trace(go.Box(y=df["log_10_minv"], name="10_minv", showlegend=False), row=4, col=2)
health_fig.add_trace(go.Scatter(x=df["dive"], y=df["depth_reached"], mode="lines", line_color="brown", name="depth_reached"), row=5, col=1)
# Battery capacity
batteryPercent_df = (df["log_AH_total_capacity"] - df["log_AH_total_consumed"])/df["log_AH_total_capacity"]
batteryUsed_thisdive_df = df["log_energy_thisdive"] # Normalize relative to 4.7Wh at 15V energy, required by the average 900m dive
health_fig.add_trace(go.Scatter(x=df["dive"], y=batteryPercent_df*100, line_color="black", mode="lines", name="batteryPercent"), row=6, col=1)
health_fig.add_trace(go.Scatter(x=df["dive"], y=batteryUsed_thisdive_df*550, mode="markers", marker_color="grey", name="dive_AHx550"), row=6, col=1)

# Update figure properties
health_fig.update_layout(height=600, width=960, title_text="<b>Health Stats</b>", title_x=0.5)

# Update x-axis properties
# mfig.update_xaxes(title_text="Dive", range=[0, 220], row=5, col=1)

# Update y-axis properties
health_fig.update_yaxes(title_text="RH%", row=1, col=1)
health_fig.update_yaxes(title_text="PSI", row=2, col=1)
health_fig.update_yaxes(title_text="°C", row=3, col=1)
health_fig.update_yaxes(title_text="V", row=4, col=1)
health_fig.update_yaxes(title_text="m", row=5, col=1)
health_fig.update_yaxes(title_text="%", row=6, col=1)

health_fig.update_xaxes(title_text="Dive", row=6, col=1)

# Remove col 2 y-axis
health_fig.update_yaxes(showticklabels=False, col=2)
# mfig.update_yaxes(showticklabels=False, row=2, col=2)
# mfig.update_yaxes(showticklabels=False, row=3, col=2)
# mfig.update_yaxes(showticklabels=False, row=4, col=2)
# mfig.update_yaxes(showticklabels=False, row=5, col=2)

health_fig.show()

# Rates

In [198]:

rates_fig = make_subplots(rows=8, cols=2, shared_xaxes=True, shared_yaxes=True, vertical_spacing=0.025, horizontal_spacing=0.01, column_widths=[0.9, 0.1])
# roll
rates_fig.add_trace(go.Scatter(x=df["dive"], y=df["roll_imax"], mode="lines", line_color="red", name="roll_imax"), row=1, col=1) # mode="lines+markers"
rates_fig.add_trace(go.Box(y=df["roll_imax"], name="", showlegend=False, line_color="red"), row=1, col=2)
rates_fig.add_trace(go.Scatter(x=df["dive"], y=df["roll_rate_min"], mode="lines",line=dict(dash='dash', color="red"), name="roll_rate_min"), row=2, col=1) # mode="lines+markers"
rates_fig.add_trace(go.Box(y=df["roll_rate_min"], name="", showlegend=False, line_color="red"), row=2, col=2)
# pitch
rates_fig.add_trace(go.Scatter(x=df["dive"], y=df["pitch_imax"], mode="lines", line_color="blue", name="pitch_imax"), row=3, col=1) # mode="lines+markers"
rates_fig.add_trace(go.Box(y=df["pitch_imax"], name="", showlegend=False, line_color="blue"), row=3, col=2)
rates_fig.add_trace(go.Scatter(x=df["dive"], y=df["pitch_rate_min"], mode="lines", line=dict(dash='dash', color="blue"), name="pitch_rate_min"), row=4, col=1) # mode="lines+markers"
rates_fig.add_trace(go.Box(y=df["pitch_rate_min"], name="", showlegend=False, line_color="blue"), row=4, col=2)
# VBD
rates_fig.add_trace(go.Scatter(x=df["dive"], y=df["vbd_i_apogee"], mode="lines", line_color="green", name="vbd_i_apogee"), row=5, col=1) # mode="lines+markers"
rates_fig.add_trace(go.Box(y=df["vbd_i_apogee"], name="", showlegend=False, line_color="green"), row=5, col=2)
rates_fig.add_trace(go.Scatter(x=df["dive"], y=df["vbd_rate_apogee"], mode="lines", line=dict(dash='dash', color="green"), name="vbd_rate_apogee"), row=6, col=1) # mode="lines+markers"
rates_fig.add_trace(go.Box(y=df["vbd_rate_apogee"], name="", showlegend=False, line_color="green"), row=6, col=2)
rates_fig.add_trace(go.Scatter(x=df["dive"], y=df["vbd_rate_min"], mode="lines", line=dict(dash='dot', color="green"), name="vbd_rate_min"), row=7, col=1) # mode="lines+markers"
rates_fig.add_trace(go.Box(y=df["vbd_rate_min"], name="", showlegend=False, line_color="green"), row=7, col=2)
# depth
rates_fig.add_trace(go.Scatter(x=df["dive"], y=df["depth_reached"], mode="lines", line_color="brown", name="depth_reached"), row=8, col=1)

rates_fig.update_yaxes(title_text="mA", row=1, col=1)
rates_fig.update_yaxes(title_text="AD/s", row=2, col=1)
rates_fig.update_yaxes(title_text="mA", row=3, col=1)
rates_fig.update_yaxes(title_text="AD/s", row=4, col=1)
rates_fig.update_yaxes(title_text="mA", row=5, col=1)
rates_fig.update_yaxes(title_text="AD/s", row=6, col=1)
rates_fig.update_yaxes(title_text="AD/s", row=7, col=1)
rates_fig.update_yaxes(title_text="m", row=8, col=1)

rates_fig.update_yaxes(showticklabels=False, col=2)

rates_fig.update_xaxes(title_text="Dive", row=8, col=1)

rates_fig.update_layout(height=800, width=960, title_text="<b>Rates Stats</b>", title_x=0.5)

rates_fig.show()


# pitch
# rates_fig.add_trace(go.Scatter(x=df["dive"], y=df["roll_imax"], mode="lines", line_color="red", name="roll_imax"), row=1, col=1) # mode="lines+markers"
# rates_fig.add_trace(go.Box(y=df["roll_imax"], name="roll_imax"), row=1, col=2)

# Calls

In [199]:
call_fig = make_subplots(rows=4, cols=2, shared_xaxes=True, shared_yaxes=True, vertical_spacing=0.01, horizontal_spacing=0.01, column_widths=[0.9, 0.1])
# calls
call_fig.add_trace(go.Bar(x=df["dive"], y=df["CALLS"], name="CALLS", marker=dict(color='red')), row=1, col=1) # mode="lines+markers"
call_fig.add_trace(go.Box(y=df["CALLS"], name="", showlegend=False, line_color="red"), row=1, col=2)
# call depth
call_fig.add_trace(go.Scatter(x=df["dive"], y=df["SM_depth"], mode="lines", line_color="blue", name="SM_depth"), row=2, col=1) # mode="lines+markers"
call_fig.add_trace(go.Box(y=df["SM_depth"], name="", showlegend=False, line_color="blue"), row=2, col=2)
# call angle
call_fig.add_trace(go.Scatter(x=df["dive"], y=df["SM_angle"], mode="lines", line_color="green", name="SM_angle"), row=3, col=1) # mode="lines+markers"
call_fig.add_trace(go.Box(y=df["SM_angle"], name="", showlegend=False, line_color="green"), row=3, col=2)
# smcc
call_fig.add_trace(go.Scatter(x=df["dive"], y=df["SM_CC"], mode="lines", line_color="black", name="SM_CC"), row=4, col=1) # mode="lines+markers"
call_fig.add_trace(go.Box(y=df["SM_CC"], name="", showlegend=False, line_color="black"), row=4, col=2)

call_fig.update_yaxes(title_text="calls", row=1, col=1)
call_fig.update_yaxes(title_text="m", row=2, col=1)
call_fig.update_yaxes(title_text="°", row=3, col=1)
call_fig.update_yaxes(title_text="SMCC", row=4, col=1)

call_fig.update_yaxes(showticklabels=False, col=2)

call_fig.update_xaxes(title_text="Dive", row=4, col=1)

call_fig.update_layout(height=600, width=960, title_text="<b>Call Stats</b>", title_x=0.5)

call_fig.show()

# Navigation

In [202]:
nav_fig = make_subplots(rows=5, cols=2, shared_xaxes=True, shared_yaxes=True, vertical_spacing=0.01, horizontal_spacing=0.01, column_widths=[0.9, 0.1])

nav_fig.add_trace(go.Scatter(x=df["dive"], y=df["MAX_BUOY"], mode="lines", line_color="red", name="MAX_BUOY"), row=1, col=1)
nav_fig.add_trace(go.Scatter(x=df["dive"], y=df["glider_dog"], mode="lines", line_color="blue", name="glider_dog"), row=2, col=1)
nav_fig.add_trace(go.Scatter(x=df["dive"], y=df["glider_sog"], mode="lines", line_color="green", name="glider_sog"), row=3, col=1)
nav_fig.add_trace(go.Scatter(x=df["dive"], y=df["dac_velocity"], mode="lines", line_color="black", name="dac_velocity"), row=4, col=1)
nav_fig.add_trace(go.Scatter(x=df["dive"], y=df["surf_velocity"], mode="lines", line_color="grey", name="surf_velocity"), row=5, col=1)

nav_fig.update_yaxes(title_text="CC", row=1, col=1)
nav_fig.update_yaxes(title_text="km", row=2, col=1)
nav_fig.update_yaxes(title_text="m/s", row=3, col=1)
nav_fig.update_yaxes(title_text="m/s", row=4, col=1)
nav_fig.update_yaxes(title_text="m/s", row=5, col=1)

nav_fig.update_yaxes(showticklabels=False, col=2)

nav_fig.update_xaxes(title_text="Dive", row=5, col=1)

nav_fig.update_layout(height=500, width=960, title_text="<b>Navigation Stats</b>", title_x=0.5)

nav_fig.show()


In [201]:
polar_fig = go.Figure()
polar_fig.add_trace(go.Scatterpolar(r=df["glider_sog"], theta=df["glider_hdg"], mode="markers", name="glider_sog", hovertext=df["dive"]))
polar_fig.add_trace(go.Scatterpolar(r=df["dac_velocity"], theta=df["dac_heading"], mode="markers", name="dac_velocity", hovertext=df["dive"]))
polar_fig.add_trace(go.Scatterpolar(r=df["surf_velocity"], theta=df["surf_heading"], mode="markers", name="surf_velocity", hovertext=df["dive"]))

polar_fig.update_layout(height=600, width=800, title_text="<b>Navigation Polar</b>", title_x=0.5)

polar_fig.show()